In [2]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.datasets import make_blobs

from sklearn.model_selection import train_test_split
from pathlib import Path
import requests

## device agnostic code

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists")
else:
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/refs/heads/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

from helper_functions import plot_predictions, accuracy_fn, plot_decision_boundary


helper_functions.py already exists


## create and visualize data

In [5]:
X_blob, y_blob = make_blobs(
                            n_samples=700,
                            n_features=2,
                            centers=7,
                            cluster_std=1.15,
                            center_box=(-10.0, 10.0),
                            shuffle=True,
                            random_state=50,
                            return_centers=False
                            )
plt.scatter(
          x=X_blob[:, 0],
          y=X_blob[:, 1],
          c=y_blob,
          cmap=plt.cm.RdYlBu
          );

## check data type

In [6]:
type(X_blob)

numpy.ndarray

## convert from nd.array to torch.tensor

In [7]:
X_blob = torch.from_numpy(X_blob).type(torch.float)
y_blob = torch.from_numpy(y_blob).type(torch.LongTensor)

In [8]:
type(X_blob)

torch.Tensor

## create train/test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
                  X_blob,
                  y_blob,
                  train_size=0.8,
                  random_state=50
                  )

## create model for data analysis

In [10]:
class BlobModelV1(nn.Module):
  def __init__(self, input_features, output_features):
    super().__init__()
    self.linear_layer_stack = nn.Sequential(
        nn.Linear(in_features=input_features, out_features=64),
        nn.Linear(in_features=64, out_features=64),
        nn.Linear(in_features=64, out_features=output_features)
    )

  def forward(self, x):
    return self.linear_layer_stack(x)

model_1 = BlobModelV1(2, 7).to(device)
model_1_str = str(model_1)
clean_model_str = model_1_str.replace("(", " ").replace(")", " ")

## define loss function and optimizer

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.01)
optimizer_class_name = optimizer.__class__.__name__
learning_rate = optimizer.param_groups[0]["lr"]

## write training and test loop

In [12]:
torch.manual_seed(50)
torch.cuda.manual_seed(50)

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

epochs_count = []
loss_values = []
test_loss_values = []
acc_values = []
test_acc_values = []

epochs = 1050

for epoch in range(epochs):
  model_1.train()
  y_logits = model_1(X_train)

  y_pred = torch.argmax(y_logits, dim=1)
  loss = loss_fn(y_logits, y_train)

  acc = accuracy_fn(y_train, y_pred)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  model_1.eval()
  with torch.inference_mode():
    test_logits = model_1(X_test)

    test_pred = torch.argmax(test_logits, dim=1)
    test_loss = loss_fn(test_logits, y_test)

    test_acc = accuracy_fn(y_test, test_pred)

    if epoch % 50 == 0:
      epochs_count.append(epoch)
      loss_values.append(loss)
      acc_values.append(acc)
      test_loss_values.append(test_loss)
      test_acc_values.append(test_acc)
      print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {acc:.2f} | Test Loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}")

Epoch: 0 | Loss: 1.76443 | Acc: 27.14 | Test Loss: 1.54501 | Test Acc: 25.71
Epoch: 50 | Loss: 0.60667 | Acc: 74.46 | Test Loss: 0.62115 | Test Acc: 77.86
Epoch: 100 | Loss: 0.50537 | Acc: 81.25 | Test Loss: 0.52970 | Test Acc: 83.57
Epoch: 150 | Loss: 0.45252 | Acc: 83.75 | Test Loss: 0.48407 | Test Acc: 85.71
Epoch: 200 | Loss: 0.41593 | Acc: 85.89 | Test Loss: 0.45403 | Test Acc: 87.14
Epoch: 250 | Loss: 0.38728 | Acc: 87.86 | Test Loss: 0.43150 | Test Acc: 87.86
Epoch: 300 | Loss: 0.36341 | Acc: 89.46 | Test Loss: 0.41330 | Test Acc: 87.14
Epoch: 350 | Loss: 0.34285 | Acc: 90.00 | Test Loss: 0.39794 | Test Acc: 87.14
Epoch: 400 | Loss: 0.32489 | Acc: 90.36 | Test Loss: 0.38467 | Test Acc: 87.86
Epoch: 450 | Loss: 0.30911 | Acc: 90.36 | Test Loss: 0.37306 | Test Acc: 88.57
Epoch: 500 | Loss: 0.29524 | Acc: 90.18 | Test Loss: 0.36286 | Test Acc: 89.29
Epoch: 550 | Loss: 0.28309 | Acc: 90.36 | Test Loss: 0.35389 | Test Acc: 89.29
Epoch: 600 | Loss: 0.27248 | Acc: 91.07 | Test Loss: 0.

## plot training | test | accuracy over epochs

In [13]:
final_loss = loss_values[-1]
final_test_loss = test_loss_values[-1]
final_acc = acc_values[-1]
final_test_acc = test_acc_values[-1]

plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.grid(True)
plt.plot(epochs_count, np.array(torch.tensor(loss_values).numpy()), label=f"Train Loss: {final_loss:.3f}")
plt.plot(epochs_count, np.array(torch.tensor(test_loss_values).numpy()), label=f"Test Loss: {final_test_loss:.3f}")
plt.title(f"\n{clean_model_str}Train vs Test Loss")
plt.xlabel(f"Epochs")
plt.ylabel("Loss")
plt.legend();
plt.subplot(1, 2, 2)
plt.grid(True)
plt.plot(epochs_count, np.array(torch.tensor(acc_values).numpy()), label=f"Train Accuracy: {final_acc:.3f}")
plt.plot(epochs_count, np.array(torch.tensor(test_acc_values).numpy()), label=f"Test Accuracy: {final_test_acc:.3f}")
plt.title(f"Loss Function:\n{loss_fn}\noptimizer: {optimizer_class_name}\nLearning Rate:{learning_rate}\n\nTrain vs Test Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend();


## plot result 

In [14]:
plt.figure(figsize=(15, 7))
plt.subplot(1, 2, 1)
plt.title("Train Result")
plot_decision_boundary(model_1, X_train, y_train)
plt.subplot(1, 2, 2)
plt.title("Test Result")
plot_decision_boundary(model_1, X_test, y_test)